In [5]:
import sys
import os
import os.path
import fnmatch
from lxml import etree
from collections import OrderedDict
import csv
import IPython
import argparse
import re
from copy import deepcopy

In [6]:
nums=['I ','V ','X ','C ','L ','I.','V.','X.','C.','L.','1','2','3','4','5','6','7','8','9','0']

In [ ]:
print('This software generates a csv with basic statistics on a selected corpus \nusage: --dir path/to/your/source/dir --csv path/to/your/target/directory/for/csv')
parser = argparse.ArgumentParser()
parser.add_argument('--dir', help= '/your/directory/to/tagged/files/')
parser.add_argument('--csv', help= '/your/directory/to/your/csv/file/')
args = parser.parse_args()

In [ ]:
path_to_folder=args.dir
files_list=fnmatch.filter(os.listdir(path_to_folder), '*.xml')

In [8]:
argsdir='/home/odysseus/Bureau/chapitres/git_repo/tagged_rest/'
argscsv='/home/odysseus/Bureau/chapitres/code/global_array/'
files_list=fnmatch.filter(os.listdir(argsdir), '*.xml')

In [17]:
def average_words_sent(tree):
    average = 0
    indexes=list()
    for sent in tree.findall(".//word[@postag='PUNsent']"):
        indexes.append(sent.getparent().index(sent))
                #print(sent.getparent().index(sent))
    words_between=list()
    for idx, index in enumerate(indexes):
        if idx == 0:
            if len(indexes) == 1:
                words_between.append(index)
            elif len(indexes) > 1:
                words_between.append(indexes[idx+1]-index-1)
        elif idx < len(indexes)-1 :
            if index<indexes[idx+1]:
                words_between.append(indexes[idx+1]-index-1)
    if len(words_between) > 0:
        average = int(sum(words_between)/len(words_between))
    else :
        print("No sentence found in this section")
    return average

In [16]:
count_header=0
with open(argscsv+'rest.csv', 'w') as f:
    for file in files_list:
        dic_stats=OrderedDict()
        tmpFile=file.replace("/",":")
        full_path=argsdir+tmpFile
        if os.path.isfile(full_path):
            tree=etree.parse(full_path)
            if tree.findall(".//div[@type='chapter']"):
                first_chap = tree.findall(".//div[@type='chapter']")[0]
                middle_chap = tree.findall(".//div[@type='chapter']")[int(len(tree.findall(".//div[@type='chapter']"))/2)]
                last_chap = tree.findall(".//div[@type='chapter']")[len(tree.findall(".//div[@type='chapter']"))-1]
            elif tree.findall(".//div[@type='book']"):
                first_chap = tree.findall(".//div[@type='book']")[0]
                middle_chap = tree.findall(".//div[@type='book']")[int(len(tree.findall(".//div[@type='book']"))/2)]
                last_chap = tree.findall(".//div[@type='book']")[len(tree.findall(".//div[@type='book']"))-1]
            elif tree.findall(".//div[@type='part']"):
                first_chap = tree.findall(".//div[@type='part']")[0]
                middle_chap = tree.findall(".//div[@type='part']")[int(len(tree.findall(".//div[@type='part']"))/2)]
                last_chap = tree.findall(".//div[@type='part']")[len(tree.findall(".//div[@type='part']"))-1]
            
            dic_stats['ref']=file
            dic_stats['title']=re.sub(u'\n','',tree.find(".//title").text).replace("     ","")
            dic_stats['author']=tree.find(".//author").attrib['name']
            dic_stats['date']=tree.find(".//date").attrib['when']
            #dic_stats['genre']=tree.find(".//term").text
            dic_stats['canon_degree']="empty"
            total_words = tree.findall(".//word")
            nb_words = len(total_words)
            dic_stats['glob_word']= nb_words
            first_ten = etree.Element("first")
            for word in total_words[0:int(((10*nb_words)/100))]:
                first_ten.append(deepcopy(word))
            last_ten = etree.Element("last")
            for word in total_words[nb_words-int(((10*nb_words)/100)):nb_words]:
                last_ten.append(deepcopy(word))
            
            dic_stats['glob_book']=len(tree.findall(".//div[@type='book']"))
            dic_stats['glob_part']=len(tree.findall(".//div[@type='part']"))
            dic_stats['glob_chapter']=len(tree.findall(".//div[@type='chapter']"))
            dic_stats['glob_paragraph']=len(tree.findall(".//p"))
            dic_stats['glob_sentence']=len(tree.findall(".//word[@postag='PUNsent']"))
            
            dic_stats['glob_av_word_per_sent']= average_words_sent(tree)
            dic_stats['glob_verb']=len(tree.findall(".//word[@postag='VERB']"))
            dic_stats['glob_adverb']=len(tree.xpath(".//word[starts-with(@postag, 'ADV')]"))
            dic_stats['glob_adj']=len(tree.xpath(".//word[starts-with(@postag, 'ADJ')]"))
            dic_stats['glob_coord']=len(tree.findall(".//word[@postag='CONJcoord']"))
            dic_stats['glob_sub']=len(tree.findall(".//word[@postag='CONJsubord']"))
            dic_stats['glob_il']=len(tree.findall(".//word[@form='il']"))
            dic_stats['glob_ils']=len(tree.findall(".//word[@form='ils']"))
            dic_stats['glob_elle']=len(tree.findall(".//word[@form='elle']"))
            dic_stats['glob_elles']=len(tree.findall(".//word[@form='elles']"))
            dic_stats['glob_je']=len(tree.findall(".//word[@form='je']"))
            dic_stats['glob_nous']=len(tree.findall(".//word[@form='nous']"))
            dic_stats['glob_etat']=len(tree.xpath(".//word[@lemma='être' or @lemma='sembler' or @lemma='devenir' or @lemma='demeurer' or @lemma='rester' ]"))
            dic_stats['glob_voc']=len(set(tree.xpath(".//word/@lemma")))
            dic_stats['first_word']=len(first_chap.findall(".//word"))
            dic_stats['first_paragraph']=len(first_chap.findall(".//p"))
            dic_stats['first_sentence']=len(first_chap.findall(".//word[@postag='PUNsent']"))
            dic_stats['first_av_word_per_sent']= average_words_sent(first_chap)
            dic_stats['first_verb']=len(first_chap.findall(".//word[@postag='VERB']"))
            dic_stats['first_adverb']=len(first_chap.xpath(".//word[starts-with(@postag, 'ADV')]"))
            dic_stats['first_adj']=len(first_chap.xpath(".//word[starts-with(@postag, 'ADJ')]"))
            dic_stats['first_coord']=len(first_chap.findall(".//word[@postag='CONJcoord']"))
            dic_stats['first_sub']=len(first_chap.findall(".//word[@postag='CONJsubord']"))
            dic_stats['first_il']=len(first_chap.findall(".//word[@form='il']"))
            dic_stats['first_ils']=len(first_chap.findall(".//word[@form='ils']"))
            dic_stats['first_elle']=len(first_chap.findall(".//word[@form='elle']"))
            dic_stats['first_elles']=len(first_chap.findall(".//word[@form='elles']"))
            dic_stats['first_je']=len(first_chap.findall(".//word[@form='je']"))
            dic_stats['first_nous']=len(first_chap.findall(".//word[@form='nous']"))
            dic_stats['first_etat']=len(first_chap.xpath(".//word[@lemma='être' or @lemma='sembler' or @lemma='devenir' or @lemma='demeurer' or @lemma='rester' ]"))
            dic_stats['first_voc']=len(set(first_chap.xpath(".//word/@lemma")))
            dic_stats['first_ten_word']=len(first_ten.findall(".//word"))
            dic_stats['first_ten_sentence']=len(first_ten.findall(".//word[@postag='PUNsent']"))
            dic_stats['first_ten_av_word_per_sent']= average_words_sent(first_ten)
            dic_stats['first_ten_verb']=len(first_ten.findall(".//word[@postag='VERB']"))
            dic_stats['first_ten_adverb']=len(first_ten.xpath(".//word[starts-with(@postag, 'ADV')]"))
            dic_stats['first_ten_adj']=len(first_ten.xpath(".//word[starts-with(@postag, 'ADJ')]"))
            dic_stats['first_ten_coord']=len(first_ten.findall(".//word[@postag='CONJcoord']"))
            dic_stats['first_ten_sub']=len(first_ten.findall(".//word[@postag='CONJsubord']"))
            dic_stats['first_ten_il']=len(first_ten.findall(".//word[@form='il']"))
            dic_stats['first_ten_ils']=len(first_ten.findall(".//word[@form='ils']"))
            dic_stats['first_ten_elle']=len(first_ten.findall(".//word[@form='elle']"))
            dic_stats['first_ten_elles']=len(first_ten.findall(".//word[@form='elles']"))
            dic_stats['first_ten_je']=len(first_ten.findall(".//word[@form='je']"))
            dic_stats['first_ten_nous']=len(first_ten.findall(".//word[@form='nous']"))
            dic_stats['first_ten_etat']=len(first_ten.xpath(".//word[@lemma='être' or @lemma='sembler' or @lemma='devenir' or @lemma='demeurer' or @lemma='rester' ]"))
            dic_stats['first_ten_voc']=len(set(first_ten.xpath(".//word/@lemma")))
            dic_stats['mid_word']=len(middle_chap.findall(".//word"))
            dic_stats['mid_paragraph']=len(middle_chap.findall(".//p"))
            dic_stats['mid_sentence']=len(middle_chap.findall(".//word[@postag='PUNsent']"))
            dic_stats['mid_av_word_per_sent']= average_words_sent(middle_chap)
            dic_stats['mid_verb']=len(middle_chap.findall(".//word[@postag='VERB']"))
            dic_stats['mid_adverb']=len(middle_chap.xpath(".//word[starts-with(@postag, 'ADV')]"))
            dic_stats['mid_adj']=len(middle_chap.xpath(".//word[starts-with(@postag, 'ADJ')]"))
            dic_stats['mid_coord']=len(middle_chap.findall(".//word[@postag='CONJcoord']"))
            dic_stats['mid_sub']=len(middle_chap.findall(".//word[@postag='CONJsubord']"))
            dic_stats['mid_il']=len(middle_chap.findall(".//word[@form='il']"))
            dic_stats['mid_ils']=len(middle_chap.findall(".//word[@form='ils']"))
            dic_stats['mid_elle']=len(middle_chap.findall(".//word[@form='elle']"))
            dic_stats['mid_elles']=len(middle_chap.findall(".//word[@form='elles']"))
            dic_stats['mid_je']=len(middle_chap.findall(".//word[@form='je']"))
            dic_stats['mid_nous']=len(middle_chap.findall(".//word[@form='nous']"))
            dic_stats['mid_etat']=len(middle_chap.xpath(".//word[@lemma='être' or @lemma='sembler' or @lemma='devenir' or @lemma='demeurer' or @lemma='rester' ]"))
            dic_stats['mid_voc']=len(set(middle_chap.xpath(".//word/@lemma")))
            dic_stats['last_word']=len(last_chap.findall(".//word"))
            dic_stats['last_paragraph']=len(last_chap.findall(".//p"))
            dic_stats['last_sentence']=len(last_chap.findall(".//word[@postag='PUNsent']"))
            dic_stats['last_av_word_per_sent']= average_words_sent(last_chap)
            dic_stats['last_verb']=len(last_chap.findall(".//word[@postag='VERB']"))
            dic_stats['last_adverb']=len(last_chap.xpath(".//word[starts-with(@postag, 'ADV')]"))
            dic_stats['last_adj']=len(last_chap.xpath(".//word[starts-with(@postag, 'ADJ')]"))
            dic_stats['last_coord']=len(last_chap.findall(".//word[@postag='CONJcoord']"))
            dic_stats['last_sub']=len(last_chap.findall(".//word[@postag='CONJsubord']"))
            dic_stats['last_il']=len(last_chap.findall(".//word[@form='il']"))
            dic_stats['last_ils']=len(last_chap.findall(".//word[@form='ils']"))
            dic_stats['last_elle']=len(last_chap.findall(".//word[@form='elle']"))
            dic_stats['last_elles']=len(last_chap.findall(".//word[@form='elles']"))
            dic_stats['last_je']=len(last_chap.findall(".//word[@form='je']"))
            dic_stats['last_nous']=len(last_chap.findall(".//word[@form='nous']"))
            dic_stats['last_etat']=len(last_chap.xpath(".//word[@lemma='être' or @lemma='sembler' or @lemma='devenir' or @lemma='demeurer' or @lemma='rester' ]"))
            dic_stats['last_voc']=len(set(last_chap.xpath(".//word/@lemma")))
            dic_stats['last_ten_word']=len(last_ten.findall(".//word"))
            dic_stats['last_ten_sentence']=len(last_ten.findall(".//word[@postag='PUNsent']"))
            dic_stats['last_ten_av_word_per_sent']= average_words_sent(last_ten)
            dic_stats['last_ten_verb']=len(last_ten.findall(".//word[@postag='VERB']"))
            dic_stats['last_ten_adverb']=len(last_ten.xpath(".//word[starts-with(@postag, 'ADV')]"))
            dic_stats['last_ten_adj']=len(last_ten.xpath(".//word[starts-with(@postag, 'ADJ')]"))
            dic_stats['last_ten_coord']=len(last_ten.findall(".//word[@postag='CONJcoord']"))
            dic_stats['last_ten_sub']=len(last_ten.findall(".//word[@postag='CONJsubord']"))
            dic_stats['last_ten_il']=len(last_ten.findall(".//word[@form='il']"))
            dic_stats['last_ten_ils']=len(last_ten.findall(".//word[@form='ils']"))
            dic_stats['last_ten_elle']=len(last_ten.findall(".//word[@form='elle']"))
            dic_stats['last_ten_elles']=len(last_ten.findall(".//word[@form='elles']"))
            dic_stats['last_ten_je']=len(last_ten.findall(".//word[@form='je']"))
            dic_stats['last_ten_nous']=len(last_ten.findall(".//word[@form='nous']"))
            dic_stats['last_ten_etat']=len(last_ten.xpath(".//word[@lemma='être' or @lemma='sembler' or @lemma='devenir' or @lemma='demeurer' or @lemma='rester' ]"))
            dic_stats['last_ten_voc']=len(set(last_ten.xpath(".//word/@lemma")))
            titles_book=0
            num_book=0
            for element in tree.findall(".//div[@type='book'][@title]"):
                if len(element.attrib["title"])>3:
                    titles_book+=1
                if any(num in element.attrib["title"] for num in nums):
                    num_book+=1
            titles_part=0
            num_part=0
            for element in tree.findall(".//div[@type='part'][@title]"):
                if len(element.attrib["title"])>3:
                    titles_part+=1
                if any(num in element.attrib["title"] for num in nums):
                    num_part+=1
            titles_chap=0
            num_chap=0
            for element in tree.findall(".//div[@type='chapter'][@title]"):
                if len(element.attrib["title"])>3:
                    titles_chap+=1
                if any(num in element.attrib["title"] for num in nums):
                    num_chap+=1
            dic_stats['titled_books']=titles_book
            dic_stats['titled_parts']=titles_part
            dic_stats['titled_chapters']=titles_chap
            dic_stats['numbered_books']=num_book
            dic_stats['numbered_parts']=num_part
            dic_stats['numbered_chapters']=num_chap
            headers=list(dic_stats.keys())
            writer = csv.DictWriter(f, delimiter=',', lineterminator='\n',fieldnames=headers)
            if (count_header==0):
                writer.writeheader()
                count_header += 1
            writer2 = csv.writer(f)
            writer2.writerow(list(dic_stats.values()))
        print('File done : '+file)

[96, 87, 31, 43, 6, 72, 45, 59, 22, 21, 17, 61, 43, 85, 67, 246, 19, 37, 36, 26, 159, 14, 119, 139, 71, 78, 88, 30, 19, 104, 167, 42, 48, 146, 17, 7, 39, 102, 25, 9, 107, 9, 37, 112, 25, 174, 51, 54, 16, 39, 105, 136, 55, 9, 6, 139, 41, 7, 62, 13, 87, 6, 9, 10, 24, 9, 9, 9, 8, 140, 132, 38, 50, 4, 37, 19, 20, 45, 43, 78, 164, 62, 52, 1, 123, 74, 127, 61, 47, 65, 51, 25, 36, 24, 102, 60, 127, 22, 45, 6, 75, 6, 7, 77, 17, 80, 29, 10, 46, 60, 120, 34, 8, 35, 21, 45, 47, 16, 47, 18, 10, 77, 38, 9, 57, 11, 14, 36, 7, 15, 11, 33, 6, 22, 12, 74, 27, 23, 12, 43, 29, 10, 14, 14, 21, 17, 61, 60, 41, 71, 70, 107, 107, 58, 15, 23, 72, 75, 76, 44, 100, 21, 21, 29, 8, 15, 6, 51, 56, 5, 35, 12, 39, 33, 80, 69, 23, 35, 75, 6, 26, 62, 71, 43, 15, 16, 26, 197, 66, 42, 53, 9, 10, 89, 31, 62, 25, 36, 58, 30, 32, 38, 17, 16, 22, 10, 14, 19, 25, 10, 17, 116, 42, 6, 50, 51, 11, 33, 7, 26, 2, 95, 28, 13, 32, 34, 50, 10, 1, 61, 35, 98, 31, 40, 30, 4, 28, 13, 89, 31, 39, 12, 54, 20, 7, 32, 30, 4, 12, 6, 15, 23,

[2, 63, 7]
24
[19, 31, 26, 36, 32, 10, 67, 28, 7, 25, 39, 36, 13, 30, 62, 11, 40, 43, 32, 35, 41, 42, 31, 12, 20, 20, 5, 6, 31, 39, 61, 64, 27, 57, 53, 31, 22, 21, 4, 37, 28, 41, 7, 42, 16, 34, 59, 30, 8, 14, 60, 3, 6, 9, 8, 19, 33, 32, 15, 41, 23, 87, 57, 33, 55, 16, 50, 26, 48, 44, 16, 29, 10, 14, 19, 33, 22, 32, 35, 6, 43, 13, 66, 47, 11, 42, 33, 43, 32, 11, 43, 25, 17, 23, 60, 22, 63, 18, 55, 10, 30, 160, 42, 67, 37, 15, 12, 21, 21, 51, 21, 22, 18, 29, 27, 36, 52, 16, 7, 21, 25, 30, 40, 19, 23, 35, 19, 49, 12, 29, 7, 14, 26, 54, 121, 71, 56, 25, 22, 22, 7, 13, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 19, 10, 27, 40, 22, 21, 30, 6, 107, 51, 24, 9, 41, 29, 14, 100, 13, 6, 31, 28, 45, 10, 8, 34, 24, 15, 22, 14, 15, 34, 24, 9, 39, 30, 24, 12, 26, 23, 6, 9, 29, 43, 27, 7, 44, 18, 39, 18, 28]
27
[12, 56, 16, 20, 32, 27, 31, 22, 13]
25
[45, -22]
11
[12, 40, 22, 75, 16, 34, 17, 29, 41, 111, 68, 111, 48, 60, 36, 51, 22, 56, 53, 50, 15, 39, 56, 74, 10, 6, 21, 16, 12, 8, 1

KeyboardInterrupt: 